In [ ]:
import sys
from pathlib import Path

sys.path.insert(0, str(Path('..').resolve() / 'src'))

import matplotlib.pyplot as plt
import pandas as pd
from fx_cookbook.spec_models import load_config
from fx_cookbook.data import load_data
from fx_cookbook.signals import compute_momentum_signal
from fx_cookbook.portfolio import build_ts_weights
from fx_cookbook.risk import estimate_covariance, compute_asset_volatility
from fx_cookbook.backtest import run_backtest, compute_metrics
from fx_cookbook.validation import run_hypothesis_test, evaluate_success_criteria, go_no_go


In [ ]:
config = load_config('../config.yaml')
df = load_data(config.data.path)
returns = df.pivot(index='date', columns='currency_pair', values='total_return').sort_index()
signal = compute_momentum_signal(returns, config.signal.lookback_min, config.signal.lookback_max, config.signal.hysteresis_threshold)
signal_pivot = signal.pivot(index='date', columns='currency', values='final_signal').sort_index()
cov = estimate_covariance(returns, config.signal.vol_decay_diagonal, config.signal.vol_decay_offdiag)
vol = compute_asset_volatility(cov)
vol_df = pd.DataFrame([vol.values] * len(returns), index=returns.index, columns=returns.columns)
weights = build_ts_weights(signal_pivot.fillna(0.0), vol_df, config.signal.max_position_pct)
costs = df.pivot(index='date', columns='currency_pair', values='bid_ask_spread').reindex(returns.index)
bt = run_backtest(weights, returns, costs)
metrics = compute_metrics(bt)
hyp = run_hypothesis_test(bt['net_return'], config.validation.alpha, config.validation.effect_size)
criteria = evaluate_success_criteria(metrics, config.validation.criteria)
verdict = go_no_go(hyp, criteria)


In [ ]:
out_dir = Path('../outputs')
fig_dir = out_dir / 'figures'
fig_dir.mkdir(parents=True, exist_ok=True)

plt.figure(figsize=(6, 3))
bt.set_index('date')['cumulative_return'].plot(title='Cumulative Net Return')
plt.tight_layout()
plt.savefig(fig_dir / 'backtest_cumulative.png', dpi=150)
plt.close()

import json
payload = {'metrics': metrics, 'hypothesis': hyp, 'criteria': criteria, 'verdict': verdict}
(out_dir / 'metrics.json').write_text(json.dumps(payload, indent=2))
summary = ['# Results Summary', f"Decision: **{verdict['decision']}**", '']
(out_dir / 'RESULTS.md').write_text('\n'.join(summary))
